In [6]:
import pandas as pd
import nltk
from nltk.tokenize import RegexpTokenizer
from collections import defaultdict
from openpyxl import load_workbook  # Import the load_workbook function

# Function to tokenize authors and create citation count dictionary
def create_citation_count(dataframe, author_column, paper_column):
    citation_count = defaultdict(lambda: defaultdict(int))
    author_frequency = defaultdict(int)  # To store the frequency of each author

    # Updated regular expression pattern to match names with spaces and initials
    tokenizer = RegexpTokenizer(r'[A-Z]\.?\s?[A-Za-z]+')

    for index, row in dataframe.iterrows():
        authors = row[author_column]
        papers = row[paper_column]
        tokens = tokenizer.tokenize(authors)

        for author in tokens:
            author_frequency[author] += 1

            # Update citation count to consider author-paper combinations
            citation_count[author][papers] += 1  # Store the full paper name

    return citation_count, author_frequency

# Replace '/content/Authors table.xlsx' with the actual path to your Excel file
file_path = 'Authors table.xlsx'

# Assuming the 'Sheet1' contains the data and 'Authors' is the author column, and 'Papers' is the paper column
sheet_name = 'Sheet1'
author_column = 'Authors'
paper_column = 'paper'

# Read the Excel file
df = pd.read_excel(file_path, sheet_name=sheet_name)

# Create a citation count dictionary and author frequency dictionary
citation_count, author_frequency = create_citation_count(df, author_column, paper_column)

# Get a set of all papers from the citation_count dictionary
all_papers = set()
for paper_freq in citation_count.values():
    all_papers.update(paper_freq.keys())

# Create a DataFrame with each author and their total count of appearances
consolidated_data = {'Author': [], 'Total Count': []}
paper_columns = [f'p{i}' for i in range(1, len(all_papers) + 1)]

for author, paper_freq in citation_count.items():
    consolidated_data['Author'].append(author)
    total_count = sum(paper_freq.values())
    consolidated_data['Total Count'].append(total_count)
    for paper in all_papers:
        consolidated_data[paper] = consolidated_data.get(paper, []) + [paper_freq.get(paper, 0)]

# Convert the list of paper counts to columns with appropriate column names
consolidated_df = pd.DataFrame(consolidated_data)
consolidated_df.columns = ['Author', 'Total Count'] + paper_columns

# Add a new column showing how many papers the author appeared in
consolidated_df['Papers Appeared In'] = (consolidated_df[paper_columns] != 0).sum(axis=1)


consolidated_df.to_excel('AuthorResults.xlsx' , sheet_name='Papers Appeared In', index=False)


# Print the consolidated DataFrame
consolidated_df


,Author,Total Count,p1,p2,p3,p4,p5,p6,p7,p8,Papers Appeared In
0,R. Baumann,1,0,0,0,1,0,0,0,0,1
1,S. Borkar,1,0,0,0,1,0,0,0,0,1
2,Hitachi,1,0,0,0,1,0,0,0,0,1
3,A. Geist,1,0,0,0,1,0,0,0,0,1
4,P. McLellan,1,0,0,0,1,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...
1066,W. Hu,1,0,0,0,0,0,0,0,1,1
1067,D. Gu,1,0,0,0,0,0,0,0,1,1
1068,P. Chan,1,0,0,0,0,0,0,0,1,1
1069,D. Yeung,1,0,0,0,0,0,0,0,1,1


In [7]:
import pandas as pd

# Assuming you have already executed the previous code to create citation_count dictionary and consolidated_df DataFrame

# Function to count the number of appearances of author1 and author2 together
def count_appearances(author1, author2, citation_count):
    papers_author1 = set(citation_count[author1].keys())
    papers_author2 = set(citation_count[author2].keys())
    common_papers = papers_author1.intersection(papers_author2)
    return len(common_papers)

# Create a DataFrame to store the count of papers each pair of authors appeared together in
author_pairs_data = []

# Loop through all unique author pairs (avoid duplicates like (author1, author2) and (author2, author1))
for i, author1 in enumerate(consolidated_df['Author']):
    for author2 in consolidated_df['Author'][i+1:]:
        count_of_papers_together = count_appearances(author1, author2, citation_count)
        author_pairs_data.append({'Author 1': author1, 'Author 2': author2, 'Count of Papers Together': count_of_papers_together})

# Create a DataFrame from the list of dictionaries
author_pairs_df = pd.DataFrame(author_pairs_data)

# Sort the DataFrame by the count of papers together in descending order
author_pairs_df = author_pairs_df.sort_values(by='Count of Papers Together', ascending=False)

# Create a Pandas Excel writer using the openpyxl engine in append mode
with pd.ExcelWriter('AuthorResults.xlsx', engine='openpyxl', mode='a') as writer:
    # Add the new sheet 'Count of Papers Together' with the author pairs data
    author_pairs_df.to_excel(writer, sheet_name='Count of Papers Together', index=False)

# Print the DataFrame showing the count of papers each pair of authors appeared together in
author_pairs_df

,Author 1,Author 2,Count of Papers Together
301774,Y. Zhang,S. A,4
18376,H. Li,S. A,4
18091,H. Li,H. Huang,4
18065,H. Li,F. Zhang,4
426915,A. J,Smola,4
...,...,...,...
265725,C. Juvekar,G. Costa,0
265724,C. Juvekar,A. Valenza,0
265723,C. Juvekar,L. Demetrio,0
265722,C. Juvekar,L. Richier,0
